# Notebook demo

This notebook contains the code accompanying the notebook demo. You can find the video [here](https://github.com/kaiko-ai/typedspark/assets/47976799/39e157c3-6db0-436a-9e72-44b2062df808).

## Set-up tables

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType, LongType, StringType
from typedspark import Column, Schema, create_empty_dataset, create_partially_filled_dataset
from datetime import datetime, timedelta

In [22]:
spark = SparkSession.builder.getOrCreate()
spark.sql("CREATE DATABASE IF NOT EXISTS vet")
spark.sql("CREATE DATABASE IF NOT EXISTS library")
spark.sql("CREATE DATABASE IF NOT EXISTS store")


DataFrame[]

In [4]:
class Appointments(Schema):
    appointment_id: Column[LongType]
    pet_id: Column[LongType]
    appointment_date: Column[DateType]
    appointment_reason: Column[StringType]
    veterinarian_name: Column[StringType]
    notes: Column[StringType]


class Pets(Schema):
    pet_id: Column[LongType]
    owner_id: Column[LongType]
    pet_name: Column[StringType]
    species: Column[StringType]
    breed: Column[StringType]
    age: Column[LongType]
    birthdate: Column[DateType]
    gender: Column[StringType]


class Vaccinations(Schema):
    vaccination_id: Column[LongType]
    pet_id: Column[LongType]
    vaccine_name: Column[StringType]
    vaccine_date: Column[DateType]
    next_due_date: Column[DateType]


class Owners(Schema):
    owner_id: Column[LongType]
    first_name: Column[StringType]
    last_name: Column[StringType]
    email: Column[StringType]
    phone_number: Column[StringType]
    address: Column[StringType]


create_empty_dataset(spark, Owners).write.saveAsTable(
    "vet.owners", format="parquet", mode="overwrite"
)
create_empty_dataset(spark, Pets).write.saveAsTable("vet.pets", format="parquet", mode="overwrite")
create_empty_dataset(spark, Appointments).write.saveAsTable(
    "vet.appointments", format="parquet", mode="overwrite"
)


In [5]:
create_partially_filled_dataset(
    spark,
    Vaccinations,
    {
        Vaccinations.vaccination_id: [1, 2, 3, 4, 5, 6, 7],
        Vaccinations.pet_id: [1, 2, 3, 1, 3, 2, 3],
        Vaccinations.vaccine_name: [
            "rabies",
            "rabies",
            "rabies",
            "lyme",
            "lyme",
            "influenza",
            "influenza",
        ],
        Vaccinations.next_due_date: [
            datetime.now() + timedelta(days=32),
            datetime.now() + timedelta(days=6),
            datetime.now() + timedelta(days=12),
            datetime.now() + timedelta(days=15),
            datetime.now() + timedelta(days=2),
            datetime.now() + timedelta(days=1),
            datetime.now() + timedelta(days=3),
        ],
        Vaccinations.vaccine_date: [
            datetime.now() + timedelta(days=32) - timedelta(days=365),
            datetime.now() + timedelta(days=6) - timedelta(days=365),
            datetime.now() + timedelta(days=12) - timedelta(days=365),
            datetime.now() + timedelta(days=15) - timedelta(days=365),
            datetime.now() + timedelta(days=2) - timedelta(days=365),
            datetime.now() + timedelta(days=1) - timedelta(days=365),
            datetime.now() + timedelta(days=3) - timedelta(days=365),
        ],
    },
).write.saveAsTable("vet.vaccinations", format="parquet", mode="overwrite")

## The demo

In [7]:
from typedspark import Catalogs, create_schema
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.getOrCreate()


In [8]:
db = Catalogs(spark)

In [9]:
vaccinations, Vaccinations = db.spark_catalog.vet.vaccinations()


In [10]:
vaccinations.show()


+--------------+------+------------+------------+-------------+
|vaccination_id|pet_id|vaccine_name|vaccine_date|next_due_date|
+--------------+------+------------+------------+-------------+
|             6|     2|   influenza|  2022-10-03|   2023-10-03|
|             7|     3|   influenza|  2022-10-05|   2023-10-05|
|             3|     3|      rabies|  2022-10-14|   2023-10-14|
|             2|     2|      rabies|  2022-10-08|   2023-10-08|
|             1|     1|      rabies|  2022-11-03|   2023-11-03|
|             5|     3|        lyme|  2022-10-04|   2023-10-04|
|             4|     1|        lyme|  2022-10-17|   2023-10-17|
+--------------+------+------------+------------+-------------+



In [11]:
Vaccinations



from pyspark.sql.types import DateType, LongType, StringType

from typedspark import Column, Schema


class Vaccinations(Schema):
    vaccination_id: Column[LongType]
    pet_id: Column[LongType]
    vaccine_name: Column[StringType]
    vaccine_date: Column[DateType]
    next_due_date: Column[DateType]

In [12]:
vaccinations.filter(Vaccinations.vaccine_name == "rabies").show()


+--------------+------+------------+------------+-------------+
|vaccination_id|pet_id|vaccine_name|vaccine_date|next_due_date|
+--------------+------+------------+------------+-------------+
|             3|     3|      rabies|  2022-10-14|   2023-10-14|
|             2|     2|      rabies|  2022-10-08|   2023-10-08|
|             1|     1|      rabies|  2022-11-03|   2023-11-03|
+--------------+------+------------+------------+-------------+



In [13]:
pivot = (
    vaccinations.groupby(Vaccinations.pet_id)
    .pivot(Vaccinations.vaccine_name.str)
    .agg(F.first(Vaccinations.next_due_date))
)

In [14]:
pivot.show()


+------+----------+----------+----------+
|pet_id| influenza|      lyme|    rabies|
+------+----------+----------+----------+
|     2|2023-10-03|      null|2023-10-08|
|     3|2023-10-05|2023-10-04|2023-10-14|
|     1|      null|2023-10-17|2023-11-03|
+------+----------+----------+----------+



In [15]:
pivot, Pivot = create_schema(pivot)


In [16]:
Pivot



from pyspark.sql.types import DateType, LongType

from typedspark import Column, Schema


class DynamicallyLoadedSchema(Schema):
    pet_id: Column[LongType]
    influenza: Column[DateType]
    lyme: Column[DateType]
    rabies: Column[DateType]

In [17]:
pivot.filter(Pivot.influenza.isNotNull()).show()


+------+----------+----------+----------+
|pet_id| influenza|      lyme|    rabies|
+------+----------+----------+----------+
|     2|2023-10-03|      null|2023-10-08|
|     3|2023-10-05|2023-10-04|2023-10-14|
+------+----------+----------+----------+



## Post-demo tear-down

In [23]:
spark.sql("DROP DATABASE vet CASCADE")
spark.sql("DROP DATABASE library CASCADE")
spark.sql("DROP DATABASE store CASCADE")


DataFrame[]